# Handelsblatt data

*Handelsblatt* is a popular German daily newspaper. According to the IVW, Informationsgemeinscahft zur Feststellung der Verbreitung von Werbeträgern (Information Community for the Assessment of the Circulation of Media), it had a circulation of 140612 daily copies in the first quarter of 2021. An appealing feature of Handelsblatt for forecasting economic activity is its focus on the economy.

We purchased Handelsblatt data from Genios, a German provider of business information. The corpus includes **980516** articles from January 1994 to November 2018. The data was purchased in February 2019. 

The data set includes 25 subfolders corresponding to a particular year (e.g., HB_1994). Each subfolder contains a few XML files which we parse to extract information relevant to our research project. Unfortunately, the copyright prevents us from publishing the data.

## Load the data

First, we need to read in the data. We create the list including the names of the 25 subfolders (`folder_list`) and apply the function `hb_load` to them in parallel by exploiting Python's `multiprocessing` library.

We extract the following XML elements:

* datum - publication date
* worte - the word count
* ressort - section/subsection of the newspaper
* titel-liste/serientitel - the name of the series
* quelle/name - the name of the newspaper
* quelle/seite-start - the page number in the newspaper
* titel-liste/titel - article's title
* titel-liste/dachzeile - article's kicker
* titel-liste/untertitel - article's subheading
* inhalt/vorspann - annotation
* inhalt/text - text of the article

In [1]:
import os

# Handelsblatt is the main folder with 25 subfolders in it
#path = 'E:\\Userhome\\mokuneva\\Handelsblatt' # your path here
path = os.getcwd().replace('\\newspaper_data_processing\\Handelsblatt', '') + '\\Handelsblatt'

# Create the list of all subfolders within Hb main folder.
folder_list=[]

for f in [f for f in os.listdir(path) ]: 
    # os.listdir(path) - names of directories                                         
    folder_list.append(path + '\\' + f)

In [2]:
import multiprocessing as mp # use multiprocessing module for parallel computing

NUM_CORE = mp.cpu_count()-8 # set the number of cores to use

print("The number of cores that will be used: {}".format(NUM_CORE))

The number of cores that will be used: 56


In [3]:
from datetime import datetime
startTime = datetime.now() # track time

import pandas as pd # load pandas: python data analysis library
import hb_load # import a function that loads the data from one folder (see hb_load.py file for details)

if __name__ == "__main__":
    pool = mp.Pool(NUM_CORE)
    data_intermediate = pool.map(hb_load.hb_load, folder_list) # load data from each folder in parallel
    data = pd.concat(data_intermediate) # concatenate DataFrames corresponding to different folders
    pool.close()
    pool.join()
    
print(datetime.now()-startTime)

0:00:39.740173


In [4]:
data = data.sort_values(['year', 'month', 'day'], ascending=[True, True, True]) # sort the data in chronological order
data.reset_index(inplace=True, drop=True) # reset the index of the DataFrame

In [5]:
data[10:15]

,year,month,day,newspaper,texts,rubrics,title,series_title,title_only,kicker,word_c,page
10,1994,1,3,Handelsblatt,Nachrichten. Yuan abgewertet dpa PEKING. China...,Nachrichten,Nachrichten.,,Nachrichten.,,68,
11,1994,1,3,Handelsblatt,Nachrichten. Ruecktritt abgelehnt afp NEU DELH...,Nachrichten,Nachrichten.,,Nachrichten.,,58,
12,1994,1,3,Handelsblatt,Nachrichten. Ungarn wertet Forint ab rtr BUDAP...,Nachrichten,Nachrichten.,,Nachrichten.,,33,
13,1994,1,3,Handelsblatt,Nachrichten. FDP zum Beamtenrecht dpa BONN. Di...,Nachrichten,Nachrichten.,,Nachrichten.,,55,
14,1994,1,3,Handelsblatt,Nachrichten. Neue Drohungen dpa HAMBURG. Der F...,Nachrichten,Nachrichten.,,Nachrichten.,,86,


In [6]:
# the number of article before pre-processing
len(data)

980516

## Light pre-processing

The first part of the article 327123 contains different symbols (e.g. zw|lf) instead of umlauts. The second part (which I keep) has the same content with the right format. The second part starts with 'NEU DELHI.'.

In [7]:
data.at[327123, 'texts'] = data['texts'][327123][data['texts'][327123].find('NEU DELHI.'):]

### Remove short articles (<100 words)

An important feature of the text is its length. There are a few ways to count the number of words in a text:

1. Use the metadata ('worte') of an XML file. Beware that numbers are counted as words. This is the column 'word_c' in the DataFrame. 
2. Quick and dirty approach: split the tokens by space and calculate the length of the list. Numbers and other non-alphabetic characters are counted as words. If a space is used as a delimiter in large numbers (100 000), this number will be counted as two tokens.
3. Use `count_words_mp` function, which counts only words. This is our preferred method because we exclude numbers from the analysis in both sentiment analysis and topic modeling.

In [8]:
# the second approach
# data['w_count'] = data['texts'].str.split(' ').str.len()

In [9]:
from datetime import datetime
startTime = datetime.now() # track time

import count_words_mp # import the function calculating the number of words in a text

if __name__ == "__main__":
    pool = mp.Pool(NUM_CORE)
    count_results = pool.map(count_words_mp.count_words_mp, [text for text in data['texts']]) 
    pool.close()
    pool.join()
    
print(datetime.now()-startTime)

0:00:20.254305


In [10]:
# Save the result as a new column "word_count"
data['word_count'] = count_results

The difference between the first ('word_c' column) and third ('word_count' column) approaches might be significant. 

In [11]:
diff = (data['word_c']-data['word_count'])

In [12]:
sorted(diff, reverse=True)[:10]

[695, 691, 591, 430, 429, 429, 404, 390, 387, 387]

This happens because the third approach does not count a number as a word.

In [13]:
data['texts'][list(diff).index(387)][:300]

'Fußball-Bundesliga. \\\\\\\\\\\\\\ zu Hause\\ auswärts\\ Sp.\\ g.\\ u.\\ v.\\ Tore\\ Diff.\\ Pkt.\\ Sp.\\ g.\\ u.\\ v.\\ Tore\\ Pkt.\\ Sp.\\ g.\\ u.\\ v.\\ Tore\\ Pkt.\\ 1. ( 1)\\ Bayern München 34\\ 20\\ 11\\ 3\\ 68:34\\ +34\\ 71\\ 17\\ 13\\ 4\\ 0\\ 37:12\\ 43\\ 17\\ 7\\ 7\\ 3\\ 31:22\\ 28\\ 2. ( 2)\\ Bayer Leverkusen 34\\ 21\\ 6\\ 7\\ 69:41\\ +28\\ 69'

Because short texts have sparse semantic features, topic models and BOW-based sentiment tools perform better on longer texts. This is why we keep the texts longer than 100 words.

However, at this point, we also keep articles shorter than 100 words if they can potentially be continued on another page and therefore represent part of the chained articles.

In [14]:
continued_strings_short = ['Fortsetzung von Seite', 'FORTSETZUNG VON SEITE', 'Fortsetzung Seite', 'FORTSETZUNG SEITE',
                           'Fortsetzung auf Seite', 'FORTSETZUNG AUF SEITE', 'Fortsetzung nächste Seite', 
                           'FORTSETZUNG NÄCHSTE SEITE']
# keep articles that contain at least one of the strings from the list 'continued_strings_short' and contain 
# less than 100 words
short_cont_ind = list(data[(data['word_count']<100) & 
    (data.texts.str.contains('|'.join(continued_strings_short)))].index)

An example of the articles we keep.

In [15]:
data.iloc[short_cont_ind[0]]['texts']

'New York: Anfangs besser. An der Wall Street eroeffnete der Dow-Jones-Index am Montag mit steigenden Notierungen, rutschte aber bis 12.45 Uhr Ortszeit auf 3752,69, 1,40 Punkte unter den Vortragsschluss. Fortsetzung Seite 17'

We also keep short articles published on the same day as articles with the `short_cont_ind` indices and with the same beginning of the title. These news reports may be part of longer chained articles as well.

In [16]:
startTime = datetime.now() # track time
# articles with this beginning of the title are not chained articles
exceptions = ['Nachrichten', 'Geldticker', 'Wirtschaft', 'Fortsetzun']
cont_index = []
for ind in short_cont_ind:
    # short articles published on the same day as articles with short_cont_ind indices
    cont_art = data[(data.day == data.day[ind]) & (data.month == data.month[ind]) & (data.year == data.year[ind]) &
                   (data['word_count']<100)]
    for i in cont_art.index:
        # articles with the same beginning of the title -> potentially chained articles
        if (data['texts'][i][:10] == data['texts'][ind][:10]) and (i!=ind) and \
        (data['texts'][ind][:10] not in exceptions):
            cont_index.append(i)
print(datetime.now()-startTime)

0:00:01.728834


Here is the first part of the chained article published on the page 15 that does not contain any string suggesting that it will be continued on another page.

In [17]:
data.iloc[cont_index[0]]['texts']

'AUSLAENDISCHE BOERSEN / Alle Indizes verbuchen Gewinne - London geschlossen. Aktienmaerkte in Europa begruessen das neue Jahr mit steigenden Notierungen. cbu DUESSELDORF. Die europaeischen Aktienmaerkte gingen sehr freundlich in das neue Jahr. Alle fuehrenden Indizes lagen im Plus, die wichtige Boerse in London blieb aber geschlossen. In Fernost erreichten Singapur und Hongkong neue Rekordstaende.'

However, we can also find the second part of this article including a string 'Continued from page 15' and having the same title as the first part.

In [18]:
data.iloc[short_cont_ind[1]]['texts']

'AUSLAENDISCHE BOERSEN / Fortsetzung von Seite 15. Weitere Rekorde in Fernost Tokio: Geschlossen. Wegen eines Feiertages blieben Behoerden, Banken und Maerkte geschlossen.'

In [19]:
short_keep = list(sorted(set(short_cont_ind + cont_index)))

In [20]:
# remove articles with the number of words<100 unless these are chained articles candidates
data = data.loc[short_keep].append(data[data['word_count']>=100])
data = data.sort_values(['year', 'month', 'day'], ascending=[True, True, True]) # sort the data in chronological order
data.reset_index(inplace=True, drop=True) # reset the index of the DataFrame

C:\Users\mokuneva\AppData\Local\Temp\2\ipykernel_31288\2052437139.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.loc[short_keep].append(data[data['word_count']>=100])


In [21]:
# the number of articles after removing short articles
len(data)

673712

### Remove exact duplicates

A few examples of duplicates in our corpus:
* The same article enters the corpus twice with different publication dates (e.g., 6.4.1994 and 27.4.1994). In this case, a natural solution is to keep the first entry.
* The same article appears twice with a slight variation in the metadata (e.g., one of the duplicates includes the title of the series, or the word count is a little different even though the articles are identical).
* The same article enters the corpus twice with the same publication date and metadata.

In [22]:
# All the duplicated articles are saved as 'hb_duplicates' for further exploration.
hb_duplicates = data[data['texts'].duplicated(keep = False)]

In [23]:
hb_duplicates[:10]

,year,month,day,newspaper,texts,rubrics,title,series_title,title_only,kicker,word_c,page,word_count
9710,1994,4,29,Handelsblatt,Importlizenzen im Vergleichsverfahren. HB DUES...,Weltwirtschaft,Importlizenzen im Vergleichsverfahren.,,Importlizenzen im Vergleichsverfahren.,,121,,112
9830,1994,5,2,Handelsblatt,Importlizenzen im Vergleichsverfahren. HB DUES...,Weltwirtschaft,Importlizenzen im Vergleichsverfahren.,,Importlizenzen im Vergleichsverfahren.,,122,,112
15338,1994,7,7,Handelsblatt,NACHTREPORT / Dow-Jones-Index steigt um 22 Pun...,Finanzzeitung; Geld und Kredit,NACHTREPORT / Dow-Jones-Index steigt um 22 Pun...,,NACHTREPORT / Dow-Jones-Index steigt um 22 Pun...,,638,,604
15598,1994,7,8,Handelsblatt,NACHTREPORT / Dow-Jones-Index steigt um 22 Pun...,Finanzzeitung; Geld und Kredit,NACHTREPORT / Dow-Jones-Index steigt um 22 Pun...,,NACHTREPORT / Dow-Jones-Index steigt um 22 Pun...,,636,,604
16977,1994,7,28,Handelsblatt,Glossar. Das Verarbeitende Gewerbe ist in West...,Wirtschaft und Politik,Glossar.,,Glossar.,,139,,130
18493,1994,8,19,Handelsblatt,Solide Basisausbildung zu DDR-Zeiten. Ost-Inge...,Karriere,Solide Basisausbildung zu DDR-Zeiten. Ost-Inge...,,Solide Basisausbildung zu DDR-Zeiten. Ost-Inge...,,176,,169
18627,1994,8,19,Handelsblatt,Solide Basisausbildung zu DDR-Zeiten. Ost-Inge...,Karriere,Solide Basisausbildung zu DDR-Zeiten. Ost-Inge...,,Solide Basisausbildung zu DDR-Zeiten. Ost-Inge...,,176,,169
20943,1994,9,20,Handelsblatt,Hochtief will seine Beteiligung bei Holzmann e...,Unternehmen und Maerkte,Hochtief will seine Beteiligung bei Holzmann e...,,Hochtief will seine Beteiligung bei Holzmann e...,,738,,715
20947,1994,9,20,Handelsblatt,Hochtief will seine Beteiligung bei Holzmann e...,Unternehmen und Maerkte,Hochtief will seine Beteiligung bei Holzmann e...,,Hochtief will seine Beteiligung bei Holzmann e...,,738,,715
21266,1994,9,22,Handelsblatt,Glossar. Das Verarbeitende Gewerbe ist in West...,Wirtschaft und Politik; Konjunkturbarometer,Glossar.,,Glossar.,,133,,126


In [24]:
# drop the exact duplicates, keep the article with the earlier publication date ('first')
data.drop_duplicates(['texts'], keep = 'first', inplace=True)
data.reset_index(inplace=True, drop=True)

In [25]:
# the number of articles after removing exact duplicates
len(data)

670961

## Filtering

### Section

Handelsblatt's articles are organized into 2073 unique sections/subsections. We investigate the most frequently met ones and remove articles published within a few of them.

In [26]:
# Sections and the number of articles per section
import collections
counter_sections = collections.Counter(data.rubrics)
print(counter_sections.most_common(10))

[('Unternehmen und Märkte', 87617), ('Wirtschaft und Politik', 33223), ('Deutschland', 29652), ('Finanzzeitung', 28435), ('Unternehmen & Märkte', 19289), ('Meinung und Analyse', 17238), ('Titelseite', 16957), ('Beilage oder Sonderseite', 16278), ('International', 16252), ('Europa', 15709)]


To ensure consistency in topical content over time, we remove articles from a few sections that were covered only within a limited time period:

* 1) 'Karriere': news related to work, career; due to the organizational changes, starting from 2014, this section is only present in a weekend edition of the newspaper.
* 2) 'Weekend Journal': the section was introduced in 2002 and did not receive any coverage after 2008. Moreover, the news discussed within this section is on average longer (663 words) than the news from the main section of interest "Wirtschaft und Politik" (436 words).
* 3) 'Panorama': news from around the world, issued in 1997, existed until 2001.
* 4) 'Business-Travel': travel-related news, e.g. weather, 2001-2006, only 366 articles in total.
* 5) 'Fortschritt': scientific findings, new devices, 2001-2003.
* 6) 'Wochenende': weekend news, 2012-2018, average article length is 1579.
* 7) 'perspektiven': perspectives, non-economic topics like a prize for women in science, working abroad, or new beauty standards popularized by Dove's advertising campaign; 2007-2009.

In [27]:
abandoned_sections = ['Karriere', 'Weekend Journal', 'Panorama', 'Business-Travel', 
                    'Fortschritt', 'Wochenende', 'perspektiven']

In [28]:
data = data[~data['rubrics'].isin(abandoned_sections)]

In [29]:
# the number of articles after removing articles from the sections covered only within a certain time period
len(data)

658760

Here we want to exclude the articles from non-economic sections. The list of irrelevant sections follows.

* 1) 'Sportreport': sport news.
* 2) 'Kunstmarkt': art market.
* 3) 'Sportreport      Nachrichten': sport news.
* 4) 'Neue Bücher': an advertisement for new books.
* 5) 'Wissenschaft & Debatte': science and debate.
* 6) 'Literatur': literature.
* 7) 'Leserbriefe': letters to the editor; starting from 2013, there are almost no letters/comments from the readers, because all the comments became digital.
* 8) 'Reisen und Tagen': travel.
* 9) 'Kultur': culture.
* 10) 'Termin- und Optionsmärkte': futures and options markets, mostly quantitative info.
* 11) 'Galerie des Stils': style and fashion.
* 12) 'Galerie': non-economic news about a pop band, Nutella restaurant etc.
* 13) 'Neue Buecher': new books.
* 14) 'Online': news about the websites, digitalization, online services.
* 15) 'Forschung und Technik': research and technology.
* 16) 'Business-Service': news unrelated to the economy, e.g. telephone service for weather forecast, travel information etc.
* 17) 'Computer und Kommunikation': computers and communication.
* 18) 'Ökonomie & Bildung': economics and education, Handelsblatt's materials for school lessons, MBA studies, educating articles about how the economy works.
* 19) 'Auto-Mobil': new car models.
* 20) 'Die Handelsblatt-Woche': announcement of events taking place this week.
* 21) 'Wirtschaftsbuch': economics books.

In [30]:
non_economic_sections = ['Kunstmarkt', 'Sportreport', 'Sportreport      Nachrichten', 
                  'Neue Bücher', 'Wissenschaft & Debatte',  
                  'Literatur', 'Leserbriefe', 'Reisen und Tagen', 
                  'Kultur',  'Termin- und Optionsmärkte',
                  'Galerie des Stils', 'Galerie', 'Neue Buecher', 'Online', 'Forschung und Technik', 
                  'Business-Service', 'Computer und Kommunikation', 'Ökonomie & Bildung', 'Auto-Mobil', 
                  'Die Handelsblatt-Woche', 'Wirtschaftsbuch']

In [31]:
data = data[~data['rubrics'].isin(non_economic_sections)]

In [32]:
# the number of articles after removing articles from the non-economic sections
len(data)

621597

Additionally, we remove articles from the related sections and subsections. 

* This means that we not only remove articles from a section 'Kunstmarkt', but also from the subsections 'Kunstmarkt      Anleihen', 'Kunstmarkt      Aus den Galerien' etc. 

* We consider 'Kunstmarkt' and 'Themen und Trends      Kunstmarkt' as two close sections. 

* The full list of sections/subsections we remove can be found in 'subsections_hb.txt'.

In [33]:
# this function loads the dictionary with the related sections and subsections we want to exclude
from os import path
import codecs

def dictionary_open(name):
    with codecs.open(path.join(os.getcwd(), name),
               'r',  'utf-8') as f:
          dictionary = set(f.read().splitlines()[1:-1])
    return dictionary

In [34]:
subsections_clean = dictionary_open('subsections_hb.txt')

In [35]:
data = data[~data['rubrics'].isin(subsections_clean)]
data.reset_index(inplace=True, drop=True)

In [36]:
# the number of articles after removing articles from the related subsections
len(data)

589117

In [37]:
# Sections and the number of articles per section
counter_sections=collections.Counter(data['rubrics'])
print(counter_sections.most_common(10))

[('Unternehmen und Märkte', 87617), ('Wirtschaft und Politik', 33223), ('Deutschland', 29652), ('Finanzzeitung', 28435), ('Unternehmen & Märkte', 19289), ('Meinung und Analyse', 17238), ('Titelseite', 16957), ('Beilage oder Sonderseite', 16278), ('International', 16252), ('Europa', 15709)]


### Section + text

Remove articles for which the following two conditions are met: a section is '' (unclassified), and a text contains a string 'KARRIERE'. As discussed, this type of articles is removed due to organizational changes (see above).

In [38]:
data = data[~((data['rubrics']=='') & (data['texts'].str.contains('KARRIERE')))]

Remove articles that are characterized by the following conditions: a section contains a string "Recht und Steuern" (Law and taxes), and a text contains one of the legal terms from the list `legal_terms`.

* Az./AZ/Aktenzeichen/AKTENZEICHEN - docket number
* LAG/Landesarbeitsgericht - Regional Labour Court
* BAG/Bundesarbeitsgericht - Federal Labour Court
* VI R/IV A - case/law reference
* BFH - Bundesfinanzhof, German Federal Fiscal court
* Abs. - section
* BGH/Bundesgerichtshof - Federal Supreme Court
* BGB - Bürgerliches Gesetzbuch, Civil Code
* BSG - Bundessozialgericht, Federal Social Court
* EStG/Einkommensteuergesetz - Income Tax Act
* OLG/Oberlandesgericht - Higher Regional Court
* Anwaltgerichtshof - Lawyers' Court
* Finanzgericht - Tax Court

We remove these articles because they contain very detailed explanations of laws/tax deduction rules/court decisions which have a low chance to be relevant for forecasting economic activity.

In [39]:
legal_terms = ['Az\\.', 'AZ', 'Az\\:', 'Aktenzeichen', 'AKTENZEICHEN', 'LAG', 'Landesarbeitsgericht', 'Bundesarbeitsgericht', 'BAG', 'Arbeitsgericht', 'VI R','IV A','BFH','Abs\\.','BGH', 'Bundesgerichtshof', 'BGB','BSG','EStG','Einkommensteuergesetz','OLG', 'Oberlandesgericht', 'Anwaltgerichtshof', 'Finanzgericht']
data = data[~((data.rubrics.str.contains('Recht und Steuern')) & (data.texts.str.contains('|'.join(legal_terms))))]

In [40]:
# the number of articles after filtering out articles based on the section and text
len(data)

584047

### Section + title

Remove articles satisfying two conditions: section is 'Inhalt' (content), and the title is not 'Termine des Tages.' Section 'Inhalt' mainly includes news reports about the Handelsblatt and its organization. The aritlces with the title 'Termine des Tages.' may include short news on the Economy.

In [41]:
data = data[~(((data['rubrics']=='Inhalt') | (data['rubrics']=='Inhalt ;') | (data['rubrics']=='Inhalt      Der Werber-Rat'))
              & (data['title']!='Termine des Tages.'))]

In [42]:
# the number of articles after removing articles from the section 'Inhalt'
len(data)

583278

### Title

Exclude articles with the following title patterns:

* book suggestions (Buchtip)
* some very short news on a few people (BUSINESS LOUNGE.)
* articles about Handelsblatt and its organization (Liebe Leser)
* articles with comments from the readers (DIE MEINUNG UNSERER LESER.)
* announcement of events taking place this week (DIE HANDELSBLATT-WOCHE.)
* non-economic news related to numbers (e.g., how many people watched the movie this week); Bilanz des Wochenendes.
* sport news (SPORT TELEGRAMM.) 
* news related to the project 'Handelsblatt macht Schule' promoting economic education

In [43]:
title_patterns = ['BUCHTIP', 'Buchtip', 'BUSINESS LOUNGE\\.', 'Liebe Leser', 'DIE MEINUNG UNSERER LESER\\.', 
                 'DIE HANDELSBLATT\\-WOCHE\\.', 'Die Handelsblatt\\-Woche\\.', 'Die Handelsblattwoche\\.',
                 'DIE HANDELSBLATTWOCHE\\.', 'Bilanz des Wochenendes\\.', 'BILANZ DES WOCHENENDES\\.',
                 'SPORT TELEGRAMM\\.']
data = data[~data['title_only'].str.contains('|'.join(title_patterns))]

kicker_title_patterns = ['Handelsblatt macht Schule', 'HANDELSBLATT MACHT SCHULE\\.', 
                         'Aktuelles Wirtschaftswissen für den Unterricht', 
                         'Wirtschaftswissen für Schüler leicht gemacht',
                        'Ökonomie ganz leicht gemacht für Jugendliche',
                        'Aktuelles Wirtschaftswissen für Jugendliche',
                        'Neuer Stoff für den Unterricht im Fach Wirtschaft',
                        'Handelsblatt bringt Praxis in die Schulen',
                        'Experten aus der Praxis in den Schulen',
                        'Der neue Newcomer ist da',
                        'Die neue Zeitung für Schüler ist da',
                        'Wettbewerb\\: Eine Welt ohne Geld\\?',
                        'Wettbewerb zum Thema Geld',
                        'Wettbewerb für Schüler',
                        'Schlauere Azubis dank Zeitunglesen',
                        'Förderung für Auszubildende'
                        ]
data = data[~data.title.str.contains('|'.join(kicker_title_patterns))]

Exclude articles with the following titles:

* Kontakte. (contacts of organizations: addresses and phone numbers).

In [44]:
data = data[~(data['title_only']=='Kontakte.')]

In [45]:
# the number of articles after excluding articles based on the title patterns
len(data)

580876

### Series

Exclude articles from the following non-economic series:

* Neue Wirtschaftsliteratur (Handelsblatt-Beilage): economic books;
* Golf (Handelsblatt-Beilage): golf;
* Literatur (Handelsblatt-Beilage): literature;
* Macher des Handelsblatts (Handelsblatt-Serie): about journalists working for Handelsblatt.

In [46]:
series_exclude = ['Neue Wirtschaftsliteratur (Handelsblatt-Beilage)', 
                  'Golf (Handelsblatt-Beilage)', 'Literatur (Handelsblatt-Beilage)', 
                  'Literatur (Handelsblatt - Beilage)', 'Macher des Handelsblatts (Handelsblatt-Serie)']
data = data[~data['series_title'].isin(series_exclude)]
data.reset_index(inplace=True, drop=True) # reset the index of the DataFrame

In [47]:
# the number of articles after excluding articles from the non-economic series
len(data)

580228

### Text

Exclude articles that contain the following strings:

* TERMINMÄRKTE / Tagesbericht: options trading, mainly quantitative information;
* Deutsche Börsen Düsseldorf: stock market, mainly quantitative information;
* DIGIX. Der E-Business-Index. or DIGIX. DIGIX Der E-Business-Index.: text corresponding to a graph.

In [48]:
text_strings = ['TERMINMÄRKTE \\/ Tagesbericht', 'TERMINMAERKTE \\/ Tagesbericht', 
                'Deutsche Börsen Düsseldorf', 'Deutsche Boersen Duesseldorf', 'DIGIX\\. Der E\\-Business\\-Index.',
               'DIGIX\\. DIGIX Der E\\-Business\\-Index\\.']
data = data[~(data.texts.str.contains('|'.join(text_strings)))]
data.reset_index(inplace=True, drop=True) # reset the index of the DataFrame

In [49]:
# the number of articles after excluding articles based on the text patterns
len(data)

580102

In [50]:
counter=collections.Counter(data['year'])
print(counter)

Counter({2000: 33949, 1999: 32861, 2001: 30730, 1996: 27326, 1997: 27086, 1998: 26912, 2004: 26432, 1995: 26170, 1994: 25349, 2002: 25083, 2005: 24726, 2003: 24350, 2006: 24167, 2007: 23816, 2010: 23811, 2008: 23520, 2009: 23329, 2011: 22196, 2012: 20667, 2013: 17506, 2014: 17397, 2015: 16297, 2016: 13102, 2017: 12152, 2018: 11168})


## Umlauts

To fix the issue with umlauts, we use the notebook `Umlauts_fix` written in Python 2. 

In [51]:
umlauts = ['ä', 'ö', 'ü', 'ß', 'Ä', 'Ö', 'Ü']
umlauts_replace = ['ae', 'oe', 'ue', 'ss', 'AE', 'OE', 'UE']

In [52]:
hb_umlauts_fix = data[(data.texts.str.contains('|'.join(umlauts_replace))) & (~data.texts.str.contains('|'.join(umlauts))) & (data.year<1999)]

In [53]:
# example of the text that we try to fix with a spellchecker
hb_umlauts_fix.texts[0]

'1994 wird ein Jahr des Wandels. Auf ein Neues!. Steuern und Beitraege steigen, staatliche Leistungen werden gesenkt - wahrlich kein berauschender Auftakt fuer das neue Jahr. Ohnehin sind die Erwartungen nicht sehr rosig. Zwar wird sich die Wirtschaft erholen. Ob es zu einem kraeftigen Aufschwung reicht, ist ungewiss. Sicher ist dagegen der Anstieg der Arbeitslosigkeit. Fuer Spannung sorgt der Wahlmarathon, der erst mit den Bundestagswahlen im Oktober entschieden wird. Kein Zweifel: 1994 wird ein Jahr der Unsicherheit und des Wandels. Ob es auch ein gutes Jahr wird, haengt davon ab, wie dieser Wandel bewaeltigt wird. 1993 war das Jahr der Krisenerkenntnis. Die Einsicht in die Kluft zwischen wirtschaftlich Machbarem und Anspruchsdenken hat erste Ansaetze zur Krisenbewaeltigung ermoeglicht. In der Tarifpolitik und bei den Sozialleistungen hat es - noch vor kurzem undenkbare - Einschnitte gegeben. Auch 1994 wird es Kuerzungen geben. Nur: Mit dem Rotstift allein kann kein Ausweg aus der Kr

In [54]:
hb_umlauts_fix.to_csv('hb_umlauts_fix.csv', encoding='utf-8-sig', sep = ';')

In [55]:
import pandas as pd
hb_umlauts_fixed = pd.read_csv('hb_umlauts_fixed.csv', encoding = 'utf-8', sep=';')

C:\Users\mokuneva\AppData\Local\Temp\2\ipykernel_31288\1128006412.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  hb_umlauts_fixed = pd.read_csv('hb_umlauts_fixed.csv', encoding = 'utf-8', sep=';')


In [56]:
data.loc[hb_umlauts_fixed['Unnamed: 0.1'], 'texts'] = hb_umlauts_fixed.texts.values

In [57]:
# fixed version
data.texts[0]

'1994 wird ein Jahr des Wandels. Auf ein Neues!. Steuern und Beiträge steigen, staatliche Leistungen werden gesenkt - wahrlich kein berauschender Auftakt für das neue Jahr. Ohnehin sind die Erwartungen nicht sehr rosig. Zwar wird sich die Wirtschaft erholen. Ob es zu einem kräftigen Aufschwung reicht, ist ungewiss. Sicher ist dagegen der Anstieg der Arbeitslosigkeit. Für Spannung sorgt der Wahlmarathon, der erst mit den Bundestagswahlen im Oktober entschieden wird. Kein Zweifel: 1994 wird ein Jahr der Unsicherheit und des Wandels. Ob es auch ein gutes Jahr wird, hängt davon ab, wie dieser Wandel bewältigt wird. 1993 war das Jahr der Krisenerkenntnis. Die Einsicht in die Kluft zwischen wirtschaftlich Machbarem und Anspruchsdenken hat erste Ansätze zur Krisenbewältigung ermöglicht. In der Tarifpolitik und bei den Sozialleistungen hat es - noch vor kurzem undenkbare - Einschnitte gegeben. Auch 1994 wird es Kürzungen geben. Nur: Mit dem Rotstift allein kann kein Ausweg aus der Krise gezeic

## Remove English articles

With the next pre-processing step, we filter out news articles written in English. To do that, we use a **langdetect** library.

In [58]:
startTime = datetime.now()

import identify_eng

if __name__ == "__main__":
    pool = mp.Pool(NUM_CORE)
    eng_results = pool.map(identify_eng.identify_eng, [text for text in data['texts']]) 
    pool.close()
    pool.join()

print(datetime.now()-startTime)

0:12:51.574581


In [59]:
data['language'] = eng_results
data = data[data.language==0]
data.reset_index(inplace=True, drop=True) # reset the index of the DataFrame

In [60]:
# the number of articles after excluding English articles
len(data)

580045

## Remove URLs

Remove URLs and html file references.
E.g.: 'de/studie99.html', 'Amazon.com', etc.

In [61]:
startTime = datetime.now()

import correct_url

if __name__ == "__main__":
    pool = mp.Pool(NUM_CORE)
    url_corrected = pool.map(correct_url.correct_url, [text for text in data['texts']]) 
    pool.close()
    pool.join()

print(datetime.now()-startTime)

0:00:41.858401


In [62]:
data['texts'] = url_corrected

## O instead of 0 problem

In some cases, Optical Character Recognition (OCR) can not distinguish between '0' and 'O' ('o'). As a result, there are tokens like '1OO'. Using regular expressions, we identify problematic tokens and replace 'O' ('o') with '0'.

In [63]:
startTime = datetime.now()

import ocr_replace

if __name__ == "__main__":
    pool = mp.Pool(NUM_CORE)
    ocr_corrected = pool.map(ocr_replace.ocr_replace, [text for text in data['texts']]) 
    pool.close()
    pool.join()

print(datetime.now()-startTime)

0:00:19.013766


In [64]:
data['texts'] = ocr_corrected

## Fixing tokens containing a number and a word

In quite a few cases, a number and a word are erroneously merged together into a single token. Splitting these tokens into two tokens helps us to deal with the following problems:

1. Hyphen-separated words will have the same spelling across the whole corpus. Examples: 20Jährige/20-Jährige, 100prozentig/100-prozentig, 30minütigen/30-minütigen etc.

2. Numbers and the names of the currencies will represent two different tokens. Examples: 100DM/100 DM, 30Euro/30 Euro.

3. Numbers and the measures of time/weight/distance etc. will be split into two tokens. Examples: 100km/100 km, 1Mill/1 Mill, 16Uhr/16 Uhr.

4. Simple mistakes will be corrected: 30bis 40/30 bis 40, 10Fahrzeuge/10 Fahrzeuge, 10und/10 und.

5. Mistakes in the beginnning of the sentences will be corrected: 10Welche/10 Welche, 8Wie/8 Wie, etc.

The most frequently met exceptions from this rule are taken into account:

1. Names of the companies/organizations: 1822direkt, 3Sat, 4MBO etc.
2. Names of smartphone/airplane/satellite models: 4S, 1B, 1k, 328Jet, 5C, 777X etc.
3. German nouns like 90er and English adjectives like 21st, 3rd.

In [65]:
startTime = datetime.now()

import split_number_word

if __name__ == "__main__":
    pool = mp.Pool(NUM_CORE)
    split_corrected = pool.map(split_number_word.split_number_word, [text for text in data['texts']]) 
    pool.close()
    pool.join()

print(datetime.now()-startTime)

0:00:19.568016


In [66]:
data['texts'] = split_corrected

## Remove fuzzy duplicates

By fuzzy duplicates we understand nearly duplicated articles. These are:

* drafts/minor revisions of the articles saved in the database;
* slightly changed advertisements which are published several times during a month;
* 'NACHT - REPORT' news reports about stock market which are published a few hours earlier than almost identical 'WALL STREET' articles.

We identify 'fuzzy' duplicates using cosine similarity and choose a threshold of 93% based on some visual exploration. Here is the article by Ryan Basques we used as a reference: [Link](https://towardsdatascience.com/a-laymans-guide-to-fuzzy-document-deduplication-a3b3cf9a05a7). 

There are also a few exceptions that we take into account:

* 'DEUTSCHE AKTIEN' articles are often erroneously classified as duplicates because they contain very similar information;
* 'Fortsetzung von Seite' (continuation of an article from another page): not surprisingly, these documents are also very close to each other. Later we concatenate them into one article;
* cosine similarity approach does not work perfectly for extremely long articles. A few of them (e.g., 'Stuttgart 21: Ist das Projekt noch zu stoppen?') we list as exceptions.

In [67]:
# Required input for the function 'fuzzy_duplicates': a dataframe for each month-year combination.
# List with a year
inputs_year = []
# List with a month
inputs_month = []
# List with the dataframes containing 'year', 'month', and 'texts' columns
inputs_month_year = []
for year in list(set(data['year'])):
    for month in list(set(data['month'])):
        inputs_year.append(year)
        inputs_month.append(month)
        inputs_month_year.append(data[(data['year'] == year) & (data['month'] == month)][["month", "year", "texts"]])
        
inputs = list(zip(inputs_year, inputs_month, inputs_month_year))

In [68]:
startTime = datetime.now() # track time

import fuzzy_duplicates # import a function that outputs the indices of fuzzy duplicates 
delete_indices = []

if __name__ == "__main__":
    pool = mp.Pool(NUM_CORE)
    # apply function to all combinations of month-year in parallel
    delete_intermediate = pool.map(fuzzy_duplicates.fuzzy_duplicates, inputs)
    delete_indices = delete_indices + delete_intermediate # create one list of indices
    pool.close()
    pool.join()
    
print(datetime.now()-startTime)

0:02:55.219889


### Duplicates exploration

In case you want to have a look at the identified duplicates, use the function 'fuzzy_duplicates_test' from the file 'fuzzy_duplicates_test_all'.

In [69]:
startTime = datetime.now() # track time

import fuzzy_duplicates_test_all 

if __name__ == "__main__":
    pool = mp.Pool(NUM_CORE)
    dup_intermediate = pool.map(fuzzy_duplicates_test_all.fuzzy_duplicates_test, inputs) 
    duplicates = pd.concat(dup_intermediate) 
    pool.close()
    pool.join()
    
print(datetime.now()-startTime)

duplicates.to_csv('duplicates.csv', encoding = 'utf-8-sig', sep = ';')

0:02:53.796095


### Drop the duplicates

In [70]:
# Free memory
inputs = None
# List of indices corresponding to the duplicated articles
delete_indices = [item for sublist in delete_indices for item in sublist]
# List of unique indices
delete_indices = list(set(delete_indices))
# Drop the fuzzy duplicates
data.drop(data.index[delete_indices], inplace = True)
data.reset_index(inplace=True, drop=True) # reset the index of the DataFrame

In [71]:
# the number of articles after removing fuzzy duplicates
len(data)

579052

In [72]:
data.to_csv('before_chained.csv', encoding = 'utf-8-sig', sep = ';')

## Concatenate chained articles

By the chained articles we mean the news reports that start on one page and continue on another page of the newspaper. Since the news database is prepared using OCR technology, each part of a chained news article represents a separate entry in the database, i.e. a separate news report.

Ideally, all the parts of a chained article must be combined into one news report. However, as nothing in the metadata helps us identify chained articles, the merging process is not straightforward. We do our best to merge at least some chained articles based on the text alone.

In [73]:
import pandas as pd
data = pd.read_csv('before_chained.csv', encoding = 'utf-8-sig', sep=';', index_col = 0, dtype = {'kicker': 'str',
                                                                                                 'page': 'str',
                                                                                                 'series_title': 'str',
                                                                                                 'rubrics': 'str'})
data.page = data.page.fillna('')
data.series_title = data.series_title.fillna('')
data.kicker = data.kicker.fillna('')
data.rubrics = data.rubrics.fillna('')

The second part of a chained article normally contains one of the following strings: 'Fortsetzung von Seite' (continuantion from page) or 'FORTSEZUNG VON SEITE'. We select the artilces that meet this critetion and try to merge them with their beginnings.

However, we also exclude the articles that contain strings from both lists, `continued_strings_1` and `continued_strings_2`. This is because these chained articles have already been merged by the Handelsblatt team.

In [74]:
continued_strings_1 = ['Fortsetzung S\\.', 'Fortsetzung auf Seite', 'Fortsetzung Seite', 'Fortsetzung nächste Seite',
                       'Fortsetzung Seiten', 'FORTSETZUNG S\\.', 'FORTSETZUNG AUF SEITE', 'FORTSETZUNG SEITE', 
                       'FORTSETZUNG NÄCHSTE SEITE', 'FORTSETZUNG SEITEN']
continued_strings_2 = ['Fortsetzung von Seite', 'FORTSETZUNG VON SEITE']

# Articles that contain one of the strings from 'continued_strings_1' and one of the strings from 
# 'continued_strings_2' are examples of chained articles that have already been merged. 
continued_articles = data[(data.texts.str.contains('|'.join(continued_strings_2))) & 
                         (~data.texts.str.contains('|'.join(continued_strings_1)))]
continued_articles = continued_articles.reset_index()

In [75]:
# There are 844 articles that could potentially be part of the chained articles.
len(continued_articles)

844

An input of the function `chained_articles` is a tuple, where the first element is a row of the dataframe corresponding to the second part of the chained article. The second element of the tuple is a dataframe containing all articles published on the same day as the second part of the chained article, i.e. all articles that could potentially be the first part of the chained article.

In [76]:
# Required input for the function 'chained_articles': a row of the 'continued_articles' df, and a subset of the
# 'data' df corresponding to the articles published on the same day as the considered chained article.

# List with the potentially chained articles
cont_input = []
# List with the dataframes containing articles published on the same day as the considered chained article
data_input = []

# To merge the articles from 'chain_exceptions' with the corresponding first parts, more complicated rules 
# are required than those we use in the funciton 'chained_articles'. Therefore, we list them as exceptions.
chain_exceptions = [139, 140, 185]
# An input for the function 'chained_articles'.
for ind in continued_articles.index:
    if ind not in chain_exceptions:
        cont_input.append(continued_articles.iloc[[ind]])
        data_input.append(data[(data.day == continued_articles['day'][ind]) & 
                      (data.month == continued_articles['month'][ind]) &
                     (data.year == continued_articles['year'][ind])])
    
inputs_cont = list(zip(cont_input, data_input))

The function `chained_articles` returns a list with two tuples. The first tuple contains two indices corresponging to the first and second part of a chained article. These two parts must be merged into one article.
    
The second tuple contains two indices as well. The first index corresponds to a part of the chained article that is a duplicate because the merged version of this chained article already exists in the database. The article with this index must be deleted. The second index corresponds to the merged version of the chained article. This article must be kept in the database.

All the rules we use to find the first part of a chained article are described in detail in the function `chained_articles`.

In [77]:
import multiprocessing as mp # use multiprocessing module for parallel computing

NUM_CORE = mp.cpu_count()-8 # set the number of cores to use

print("The number of cores that will be used: {}".format(NUM_CORE))

from datetime import datetime

The number of cores that will be used: 56


In [78]:
startTime = datetime.now() # track time

import chained_articles # import a function that outputs two parts of a chained article and duplicated articles
chained_pair = []

if __name__ == "__main__":
    pool = mp.Pool(NUM_CORE)
    chained_intermediate = pool.map(chained_articles.chained_articles, inputs_cont)
    chained_pair = chained_pair + chained_intermediate 
    pool.close()
    pool.join()
    
print(datetime.now()-startTime)

0:00:07.383422


In [79]:
# The list of unique tuples, where the first element is a duplicate and 
# the second element is a merged chained article.
duplicates = list(set([sublist[1] for sublist in chained_pair if sublist[1] != (-1, -1)]))
# The list with indices of all the duplicated/merged articles.
dup_merged = [tup[0] for tup in duplicates]+[tup[1] for tup in duplicates]
# The list with all the chained articles that we would like to merge.
chained_pair_list = [sublist[0] for sublist in chained_pair if sublist[0] != ()]
# Exclude duplicated/merged articles from the list of the chained articles.
chained_pair_list = [pair for pair in chained_pair_list if (pair[0] not in dup_merged) and \
                     (pair[1] not in dup_merged)]
# The rules we use allow us to merge 206 chained articles
len(chained_pair_list)

206

In [80]:
# An example of the first part
data['texts'][chained_pair_list[0][0]]

'AUSLÄNDISCHE BÖRSEN / Alle Indizes verbuchen Gewinne - London geschlossen. Aktienmärkte in Europa begrüssen das neue Jahr mit steigenden Notierungen. cbu DÜSSELDORF. Die europäischen Aktienmärkte gingen sehr freundlich in das neue Jahr. Alle führenden Indizes lagen im Plus, die wichtige Börse in London blieb aber geschlossen. In Fernost erreichten Singapur und Hongkong neue Rekordstände.'

In [81]:
# An example of the second part
data['texts'][chained_pair_list[0][1]]

'AUSLÄNDISCHE BÖRSEN / Fortsetzung von Seite 15. Weitere Rekorde in Fernost Tokio: Geschlossen. Wegen eines Feiertages blieben Behörden, Banken und Märkte geschlossen.'

In [82]:
# The indices of the first part of the chained articles.
first_part_ind = [pair[0] for pair in chained_pair_list]
# The indices of the second part of the chained articles.
second_part_ind = [pair[1] for pair in chained_pair_list]
# Consider the case where the same article was paired with two different articles.
common_ind = [ind for ind in first_part_ind if ind in second_part_ind]
# In this case remove one of the pairs.
chained_pair_list = [pair for pair in chained_pair_list if pair[0] not in common_ind]
# The number of chained articles we are able to concatenate.
len(chained_pair_list)

203

In [83]:
# The indices of the first part of the chained articles.
first_part_ind = [pair[0] for pair in chained_pair_list]
# The indices of the second part of the chained articles.
second_part_ind = [pair[1] for pair in chained_pair_list]
# Merge the first and the second parts.
data.loc[first_part_ind, 'texts'] = data.loc[first_part_ind, 'texts'].values + \
" " + data.loc[second_part_ind, 'texts'].values
# Drop the second part.
data.drop(data.index[second_part_ind], inplace = True)
# The indices of the duplicates.
dup_indices = [tup[0] for tup in duplicates]
# Drop the duplicates.
data.drop(data.index[dup_indices], inplace = True)

In [84]:
import count_words_mp # import the function calculating the number of words in a text
# Calculate the word count of the merged articles.

startTime = datetime.now() # track time

if __name__ == "__main__":
    pool = mp.Pool(NUM_CORE)
    count_results = pool.map(count_words_mp.count_words_mp, [text for text in data.loc[first_part_ind, 'texts']]) 
    pool.close()
    pool.join()
    
print(datetime.now()-startTime)

0:00:01.022448


In [85]:
# Replace the "word_count" of the merged articles.
data.loc[first_part_ind, 'word_count'] = count_results
# Drop short articles.
data = data[data['word_count']>=100]
data.reset_index(inplace=True, drop=True) # reset the index of the DataFrame

In [86]:
# the number of articles after concatenating chained articles
len(data)

578620

## Handelsblatt-specific problems

There are a few small problems that are specific to Handelsblatt and that we have noticed while fixing other issues. 

&nbsp; &nbsp; 1. The end of the text with the index 205950 contains unusual symbols. We have removed this part from the text.

In [87]:
# This is the part we delete.
data['texts'][205950][data['texts'][205950].find(' k{'):]

'l'

In [88]:
data.at[205950, 'texts'] = data['texts'][205950][:data['texts'][205950].find(' k{')]

&nbsp; &nbsp; 2. Some texts contain unicode errors. For example, the name Jarosław is written as JarosÅ‚aw. We fix these errors using the function `unicode_correct`.

In [89]:
import unicode_correct # import the function correcting the unicode errors present in the database

startTime = datetime.now() # track time

to_correct = list(data[data.texts.str.contains('Å')].index)

if __name__ == "__main__":
    pool = mp.Pool(NUM_CORE)
    unicode_corrected = pool.map(unicode_correct.unicode_correct, [text for text in data.loc[to_correct, 'texts']]) 
    pool.close()
    pool.join()
    
print(datetime.now()-startTime)

# Replace the texts with the corrected version of the articles.
data.loc[to_correct, 'texts'] = unicode_corrected

0:00:02.025037


&nbsp; &nbsp; 3. We drop two texts that contain a string 'MMHANDELSBLATT' because they include a lot of errors.

In [90]:
data['texts'][71133][data['texts'][71133].find('MMHANDELSBLATT'):(data['texts'][71133].find('MMHANDELSBLATT')+1000)]

''

In [91]:
indices_drop = list(data[data.texts.str.contains('MMHANDELSBLATT')].index)
data.drop(data.index[indices_drop], inplace = True)
data.reset_index(inplace=True, drop=True) # reset the index of the DataFrame

In [92]:
# the number of articles after excluding articles that contain a string 'MMHANDELSBLATT'
len(data)

578618

&nbsp; &nbsp; 4. We drop 12 texts with broken umlauts and too many non-systematic mistakes.

In [93]:
startTime = datetime.now()

# Find the words like '}ber' and 'gel|st' in the articles' texts.
import find_umlaut

if __name__ == "__main__":
    pool = mp.Pool(NUM_CORE)
    problem_umlaut = pool.map(find_umlaut.find_umlaut, [text for text in data['texts']]) 
    pool.close()
    pool.join()

print(datetime.now()-startTime)

0:00:18.919588


In [94]:
# Indices of potentially problematic articles.
problem_umlaut_list = [tup[0] for tup in enumerate(problem_umlaut) if tup[1] != []]

In [95]:
# The last five articles contain unusual words with '|' inside. However, in these cases, '|' does not correspond
# to the umlaut 'ö'. Example: 'netzwerk|nordbayern'.
data.drop(data.index[problem_umlaut_list[:-5]], inplace = True)
data.reset_index(inplace=True, drop=True) # reset the index of the DataFrame

In [96]:
# the number of articles after excluding articles with broken umlauts and too many non-systematic mistakes
len(data)

578606

&nbsp; &nbsp; 5. Drop the text with an index 43024 because it is written in French (we have tested that all the other articles are in German).

In [97]:
data.drop(data.index[43024], inplace = True)
data.reset_index(inplace=True, drop=True) # reset the index of the DataFrame

In [98]:
# the number of articles after excluding the article in French
len(data)

578605

## Clean articles

Handelsblatt articles include some text passages that are unlikely to be relevant for either topic modeling or sentiment analysis. We decided to clean the affected articles from these text passages to make the analysis easier for our models.

We remove the following information from the texts:

   * 1) additional information in the text meant for the author
   * 2) the list of articles on a particular topic
   * 3) reference to additional information
   * 4) the calendar of events
   * 5) the composition of an index
   * 6) the box with additional information on the topic
   * 7) the addresses/Internet addresses
   * 8) the links
   * 9) the word game for investors
   * 10) information on how special pages on the topic can be ordered
   * 11) educational articles on how to understand financial news
   * 12) announcement of premieres
   * 13) announcement of trade fairs
   * 14) announcement of events
   * 15) place and time/organizer of the event

In [99]:
startTime = datetime.now()

import clean_hb_articles
if __name__ == "__main__":
    pool = mp.Pool(NUM_CORE)
    cleaned_articles = pool.map(clean_hb_articles.clean_hb_articles, [text for text in data['texts']]) 
    pool.close()
    pool.join()
    
print(datetime.now()-startTime)

0:01:58.302403


In [100]:
data['texts'] = cleaned_articles

In [101]:
import count_words_mp # import the function calculating the number of words in a text
# Calculate an updated word count of the cleaned articles.

startTime = datetime.now() # track time

if __name__ == "__main__":
    pool = mp.Pool(NUM_CORE)
    count_results = pool.map(count_words_mp.count_words_mp, [text for text in data['texts']]) 
    pool.close()
    pool.join()
    
print(datetime.now()-startTime)

0:00:22.909384


In [102]:
# Update the word count in the data frame.
data['word_count'] = count_results
# Drop short articles.
data = data[data['word_count']>=100]
data.reset_index(inplace=True, drop=True) # reset the index of the DataFrame

In [103]:
# the number of articles after excluding articles shorter than 100 words
len(data)

578392

## Exclude tables

In this section, we preprocess the news articles by removing tables to minimize noise and emphasize relevant content. We begin by calculating a numerical density metric for each text, which is computed as the ratio of the count of numbers to the total word count (excluding numbers). Texts with a numerical density of at least 20% are considered as candidates for containing tables. We manually examine these texts to identify recurring strings that typically precede tables. Using regular expressions, we exclude the tables based on these strings. Moreover, we delete some text segments predominantly comprising numbers.

In [104]:
# use the 'numeric_articles' function to identify articles with a high share of numbers in them
import sys
import os
import itertools
sys.path.insert(1, os.getcwd().replace('Handelsblatt', 'dpa'))
import numeric_articles

inputs = zip(data['texts'], data['word_count'], itertools.repeat(0.20))

startTime = datetime.now()

if __name__ == "__main__":
    pool = mp.Pool(NUM_CORE)
    tables = pool.starmap(numeric_articles.numeric_articles, inputs)
    pool.close()
    pool.join()

print(datetime.now()-startTime)
data['tables'] = tables
tables = data[data.tables == True]

0:00:13.328351


Below is an example of an article containing a table with details on new bond issuances in the Euromarket by various financial institutions. To remove this table, we apply the following regular expression:  `r'Londoner Banken berichteten von folgenden neuen Euromarkt.{0,}'`.

In [105]:
tables.loc[215,'texts']

'ANLEIHEN / Die Kapitalnachfrage ist weiter sehr hoch - Anleger sind zurückhaltend. Bayernhypo und DSL am DM - Markt. ret FRANKFURT / M. Auch zur Wochenmitte sahen sich die Bondmärkte mit zahlreichen Kapitalwünschen konfrontiert. Die anhaltend hohe Kapitalnachfrage war eine Ursache für die Zurückhaltung der Anleger und für die in diesem Kontext festzustellende anhaltende Korrekturphase an den Anleihemärkten rund um den Globus. Auch der deutsche Rentenmarkt neigte leicht zur Schwäche. Börsennotierte öffentliche DM - Anleihen wiesen Kursveränderungen zwischen plus 0,10 DM und minus 0,35 DM auf. Die Umlaufrendite erhöhte sich um einen Basispunkt auf 5,44%. Die den Markt regulierenden Stellen gaben Material im Nennwert von 38,9 Mill. DM in Frankfurt an den Markt ab; davon entfielen 20 Mill. DM auf die 61 / 4% ige Silvesteranleihe des Bundes, die bei einem Emissionskurs von 100,40% zu 99,68% in den amtlichen Handel eingeführt wurde. An der Düsseldorfer Börse belief sich der Abgabesaldo auf 

Clean tables using regular expressions.

In [106]:
import clean_tables_hb
startTime = datetime.now()
if __name__ == "__main__":
    pool = mp.Pool(NUM_CORE)
    remove_tables = pool.map(clean_tables_hb.clean_tables_hb, [text for text in data['texts']]) 
    pool.close()
    pool.join()
    
print(datetime.now()-startTime)

data['texts'] = remove_tables

0:01:01.384302


In [107]:
startTime = datetime.now()

if __name__ == "__main__":
    pool = mp.Pool(NUM_CORE)
    count_results = pool.map(count_words_mp.count_words_mp, [text for text in data['texts']]) 
    pool.close()
    pool.join()
    
print(datetime.now()-startTime)

0:00:21.745026


In [108]:
# Update the content of the column "word_count"
data['word_count'] = count_results

In [109]:
# remove articles with less than 100 words
data = data[data['word_count']>=100]
data = data.sort_values(['year', 'month', 'day'], ascending=[True, True, True]) # sort the data in chronological order
data.reset_index(inplace=True, drop=True)
# the number of articles after excluding articles shorter than 100 words
print(len(data))
del data['tables']

578168


## Identify articles that predominantly consist of numbers

Articles that predominantly consist of numerical data present challenges for sentiment or topic analysis as, once numbers are removed, they tend to contain limited information. Therefore, we remove articles with a numerical density of 50% or greater.

In [110]:
# use the 'numeric_articles' function to identify economic articles with a high share of numbers in them
inputs = zip(data['texts'], data['word_count'], itertools.repeat(0.50))

startTime = datetime.now()

if __name__ == "__main__":
    pool = mp.Pool(NUM_CORE)
    numeric_list = pool.starmap(numeric_articles.numeric_articles, inputs)
    pool.close()
    pool.join()

print(datetime.now()-startTime)
data['numeric'] = numeric_list

0:00:14.238645


In [111]:
# inspect example article with high share of numbers
data[data.numeric == True]['texts'].iloc[0]

'Liste der NBL - Beauftragten. Manfred Baatsch Generalbevollmächtigter Klöckner - Humboldt - Deutz AG Deutz - Mülheimer - Straße 111 51063 Köln Telefon: 0221 / 8222090 Telefax: 0221 / 8222095 Dr. Pol Bamelis Mitglied des Vorstandes Bayer AG 51368 Leverkusen Telefon: 0214 / 3081841 Telefax: 0214 / 3050720 Dr. Karl Braunhofer Mitglied des Vorstandes BayWa AG Arabellastrasse4, 81925 München Telefon: 089 / 9222 - 3250 Telefax: 089 / 9222 - 3703 Bruno Überredender Leiter der Materialwirtschaft Siemag Transplan GmbH Obere Industriestrasse8 57250 Netphen Telefon: 02738 / 21 - 0 Telefax: 02738 / 21 - 297 Manfred Ciesielski Mitglied des Vorstandes Karstadt AG Theodor - Althoff - Strasse2 45133 Essen Telefon: 0201 / 7271 Telefax: 0201 / 7274958 Carl Tönen Leiter des Einkaufs Deutsche Shell AG Postfach 600520, Überseering 35 22297 Hamburg Telefon: 040 / 6324 - 5800 Telefax: 040 / 6324 - 5822 Rolf Dill Ostbeauftragter Tengelmann Warenhandels sellschaft Wissollstrasse5, 45478 Mühlheim Telefon: 0208

In [112]:
data[data.numeric == True]['texts'].iloc[1]

'WAHLERGEBNISSE / In einigen Ländern drastische Denkzettel für die regierenden Kräfte. Verschiebungen in der Parteienlandschaft. HB DÜSSELDORF. Die Europawahlen brachten deutliche Verschiebungen bei den Stimmenanteilen gegenüber 1989. Die Wahlbeteiligung lag bei 57% (1989: 58,5%). Deutschland: Wahlbeteiligung 60,1%. 99 Sitze. CDU / CSU 38,8% (47 Sitze, 1989: 31 Sitze), SPD 32,2% (40 Sitze, 1989: 30 Sitze), Bündnis 90 / Die Grünen 10,1% (12 Sitze, 1989: sieben), FDP 4,1% (keinen Sitz), Republikaner 3,9% (keinen Sitz). Vorläufiges amtliches Endergebnis. Frankreich: Wahlbeteiligung: 53,5%. 87 Sitze. Neogaullisten und liberale UdF 25,5% (29 Sitze, 1989: 33 Sitze), Sozialisten 14,5% (16 Sitze, 1989: 22), Maastricht - Gegner 12,4% (13 Sitze, 1989: keinen), Nationale Front 10,6% (zehn Sitze, 1989: zehn), Kommunisten 6,9% (sechs Sitze, 1989: sieben). Vorläufiges amtliches Endergebnis. Großbritannien: Wahlbeteiligung 36%. 87 Sitze. Konservative 28% (18 Sitze, 1989: 32), Labour - Partei 45% (62 

In [113]:
len(data[data.numeric == True])

14

In [114]:
# drop articles predominantly consisting of numbers
data = data[data.numeric == False]
del data['numeric']
data.reset_index(inplace=True, drop=True)
# the number of articles after removing articles that predominantly consist of numbers
print(len(data))

578154


## Identify articles that predominantly consist of names

We eliminate texts with a name density of at least 15% (relative to the total word count, excluding numbers) as part of our pre-processing pipeline. This exclusion is important to guarantee that the remaining articles contain sufficient content for effective topic analysis, as the removal of common German names is a standard pre-processing step in LDA model estimation.

In [115]:
path = os.getcwd().replace('\\Handelsblatt', '') + '\\SZ'

# Load the dictionary containing common German first and last names
with open(path + "\\names.txt", "r", encoding="utf-8-sig") as f:
    names_list = f.read().splitlines()

In [116]:
sys.path.insert(1, os.getcwd().replace('Handelsblatt', 'SZ'))
import count_names

inputs = zip(data['texts'], data['word_count'], itertools.repeat(names_list))

startTime = datetime.now()

if __name__ == "__main__":
    pool = mp.Pool(NUM_CORE)
    names_result = pool.starmap(count_names.count_names, inputs)
    pool.close()
    pool.join()

print(datetime.now()-startTime)

data['names'] = names_result

0:01:15.355966


In [117]:
# Example of an article with a high proportion of names
data[data.names>=0.15].iloc[0]['texts']

'" wir. " . steht als Berufsverband Unternehmen der deutschen Wirtschaft unabhängig von der Unternehmensgröße und der Rechtsform für eine Mitgliedschaft offen. wir. - Vereinsrechtl. Vorstand: Dr. - Ing. E.h. Eberhard v. Künheim (Vors.) Vorsitzender des AR der BMW AG Dr. Wolfgang Söller Vorsitzender des AR Dresdner Bank AG Dr. Ludolf v. Wartenberg Hauptgeschäftsführer und Mitglied des Präsidiums des BDI e.V. Mitglieder Dr. Henning Schulte - Hölle, Allianz Holding AG Dr. Eberhard v. Körner, Asea Brown Boveri AG Dr. Jürgen Strube, BASF AG Dr. Manfred Schneider, Bayer AG Dr. Eberhard Martini, Bayerische Hypotheken - u. Wechsel - Bank AG Dr. Albrecht Schmidt, Bayerische Vereinsbank AG W. Steinriede, Berliner Bank AG Dr. Mark Wössner, Unternehmensgruppe Bertelsmann AG Dr. Christian Roth, Bilfinger & Berger Bau AG Dr. Eberh. v. Künheim, BMW AG Dr. Ludolf v. Wartenberg, Bundesverband d. Deutschen Industrie e.V. Dr. Manfred Weber, Bundesverband deutscher Banken e.V. Dr. Hubert Burda, Burda GmbH

In [118]:
# Exclude texts with a name density of at least 15%
data = data[data.names<0.15]
del data['names']
data.reset_index(inplace=True, drop=True)
# the number of articles after removing articles that predominantly consist of names
print(len(data))

578127


In [119]:
counter=collections.Counter(data['year'])
print(counter)

Counter({2000: 33869, 1999: 32788, 2001: 30645, 1996: 27152, 1997: 26929, 1998: 26855, 2004: 26417, 1995: 25944, 2002: 24961, 1994: 24737, 2005: 24702, 2003: 24311, 2006: 24128, 2007: 23795, 2010: 23771, 2008: 23498, 2009: 23309, 2011: 22175, 2012: 20626, 2013: 17475, 2014: 17390, 2015: 16280, 2016: 13094, 2017: 12144, 2018: 11132})


In [120]:
data.to_csv('hb_prepro_final.csv', encoding = 'utf-8-sig', sep = ';')